In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_pickle("../grab-ai-safety-data/df_full.pickle")

In [3]:
np.round(df.describe())

,bookingid,accuracy,bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,speed,label
count,1.450499e+07,14504993.0,14504993.0,14504993.0,14504993.0,14504993.0,14504993.0,14504993.0,14504993.0,14504993.0,14504993.0,14504993.0
mean,8.184243e+11,12.0,169.0,0.0,4.0,1.0,-0.0,0.0,-0.0,602.0,9.0,0.0
std,4.951699e+11,87.0,107.0,1.0,8.0,3.0,0.0,0.0,0.0,463.0,8.0,0.0
min,0.000000e+00,1.0,0.0,-78.0,-73.0,-76.0,-48.0,-75.0,-37.0,0.0,-2.0,0.0
25%,3.779571e+11,4.0,78.0,-1.0,-2.0,-1.0,-0.0,-0.0,-0.0,241.0,1.0,0.0
50%,8.074539e+11,4.0,169.0,0.0,9.0,1.0,-0.0,0.0,-0.0,520.0,8.0,0.0
75%,1.254130e+12,8.0,263.0,1.0,10.0,3.0,0.0,0.0,0.0,862.0,15.0,1.0
max,1.709397e+12,6070.0,360.0,67.0,75.0,78.0,40.0,80.0,66.0,8235.0,148.0,1.0


In [4]:
mask = df["bookingid"] != df["bookingid"].shift(1)

df["change_in_second"] = df["second"].diff().fillna(0)
df["change_in_bearing"] = np.abs(df["bearing"].diff().fillna(0))
df["change_in_speed"] = np.abs(df["speed"].diff().fillna(0))
df["change_in_acx"] = df["acceleration_x"].diff().fillna(0)
df["change_in_acy"] = df["acceleration_y"].diff().fillna(0)
df["change_in_acz"] = df["acceleration_z"].diff().fillna(0)
df["change_in_gyx"] = df["gyro_x"].diff().fillna(0)
df["change_in_gyy"] = df["gyro_y"].diff().fillna(0)
df["change_in_gyz"] = df["gyro_z"].diff().fillna(0)

df.loc[ mask, [
    "change_in_second", 
    "change_in_bearing",
    "change_in_speed",
    "change_in_acx",
    "change_in_acy",
    "change_in_acz",
    "change_in_gyx",
    "change_in_gyy",
    "change_in_gyz"
]] = 0

## Duration of trip
- Quite an obvious difference in trip durations after removing the outliers. Seems that shorter trips tend to be labelled safe.

In [5]:
df.groupby("bookingid").agg(
    {
        "second" : np.max,
        "label" : np.max
    }
).groupby("label").agg(
    {
    "second" : [lambda x: np.percentile(x, q=50), np.mean]
    }
)

second             
      <lambda>         mean
label                      
0        779.0   833.135876
1       1123.0  1173.834236

In [6]:
df_time = df.groupby("bookingid").agg(
    {
        "second" : np.max,
        "label" : np.max
    }
)

In [7]:
df_time.sort_values("second")

,second,label
bookingid,,
1185410973727,118.0,0
1228360646706,118.0,0
180388626478,118.0,0
34359738486,118.0,0
149,118.0,0
1374389534771,118.0,0
42949672982,118.0,0
146028888071,119.0,0
146028888115,119.0,0


## Distance of trip
- There is a weird entry with a change in time at the last entry
- Looks like there are 5 trips with ridiculous trip durations

In [8]:
df_dist = df.copy()
df_dist["distance_covered"] = df_dist["change_in_second"] * df_dist["speed"]
df_dist.groupby("bookingid").agg(
    {
        "distance_covered" : np.sum,
        "label" : np.max
    }
).groupby("label").mean()

,distance_covered
label,
0,7083.798440
1,8931.852083


In [9]:
df_dist.loc[df_dist["bookingid"] == 1503238553722, :][list(df.columns)]["second"].max()

404.0

In [10]:
duration_check = df_dist.groupby("bookingid").agg(
    {
    "second" : np.max,
    "label" : np.max
    }
)

In [11]:
duration_check.loc[duration_check["label"] == 0, :] \
              .groupby("second") \
              .count().sort_values("second", ascending=False).reset_index()["second"][1]

3614.0

## Acceleration
- Let's try euclidean first to see if that works
    - Mean of bookings, mean of label: no significant difference ~1% higher only
    - Max of the bookings, mean of label: ~12% higher for dangerous trips

In [12]:
df_acc = df.copy()
df_acc["acceleration"] = np.sqrt(
    (df_acc["acceleration_x"] ** 2) + (df_acc["acceleration_y"] ** 2) + (df_acc["acceleration_z"] ** 2)
)
test = df_acc.groupby("bookingid").agg(
    {
        "acceleration" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

,acceleration,acceleration
label,,
0,13.800160,9.872149
1,15.523679,9.904714


## Gyro
- Same technique with gyro to see
    - Max of bookings, mean of label: ~47% higher for dangerous trips
    - Mean of bookings, mean of label: ~36% higher for dangerous trips

In [13]:
df_gyro = df.copy()
df_gyro["gyro"] = np.sqrt(
    (df_gyro["gyro_x"] ** 2) + (df_gyro["gyro_y"] ** 2) + (df_gyro["gyro_z"] ** 2)
)
test = df_gyro.groupby("bookingid").agg(
    {
        "gyro" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

,gyro,gyro
label,,
0,1.38042,0.118911
1,2.04559,0.158299


## Speed
- More variations for speed
    - Max of bookings, mean of label: 5% higher for dangerous trips
    - Mean of bookings, mean of label: 10% lower for dangerous trips

In [14]:
df_spd = df.copy()
test = df_spd.groupby("bookingid").agg(
    {
        "speed" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

,speed,speed
label,,
0,20.853004,8.606909
1,21.226834,7.882894


## Bearing
- Change in bearing?